# Import Dependencies and Dataset

In [1]:
#import dependencies
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

In [2]:
# fetch dataset 
online_retail = fetch_ucirepo(id=352) 
  
# data (as pandas dataframes) 
X = online_retail.data.features
y=online_retail.data.ids

# Initial Analysis

In [ ]:
# metadata 
#online_retail.metadata

In [3]:
#Variable information
pd.set_option('display.max_colwidth', None)
online_retail.variables

,name,role,type,demographic,description,units,missing_values
0,InvoiceNo,ID,Categorical,None,"a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation",None,no
1,StockCode,ID,Categorical,None,a 5-digit integral number uniquely assigned to each distinct product,None,no
2,Description,Feature,Categorical,None,product name,None,no
3,Quantity,Feature,Integer,None,the quantities of each product (item) per transaction,None,no
4,InvoiceDate,Feature,Date,None,the day and time when each transaction was generated,None,no
5,UnitPrice,Feature,Continuous,None,product price per unit,sterling,no
6,CustomerID,Feature,Categorical,None,a 5-digit integral number uniquely assigned to each customer,None,no
7,Country,Feature,Categorical,None,the name of the country where each customer resides,None,no


In [4]:
#Combine IDs and Features
online_retail_trans = pd.concat([X, y], axis=1)

In [5]:
#Look at first five rows 
online_retail_trans.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNo,StockCode
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,536365,85123A
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,71053
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,536365,84406B
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029G
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029E


In [6]:
#Rows and column count in dataset
num_rows, num_columns = online_retail_trans.shape
print(f"The number of rows in the dataset is {num_rows} and the number of columns is {num_columns}")


The number of rows in the dataset is 541909 and the number of columns is 8


# Data Analysis and Cleanup


In [7]:
#Metadata says that rows where InvoiceNo starts with C are cancelled orders. Create two datasets, one with orders and one with cancelled orders

# Orders Dataset
orders_df = online_retail_trans[~online_retail_trans['InvoiceNo'].str.startswith('C')].copy()
#Cancelled Orders Dataset
cancelled_orders_df = online_retail_trans[online_retail_trans['InvoiceNo'].str.startswith('C')].copy()

# Display the filtered DataFrame
print(orders_df.shape)
print(cancelled_orders_df.shape)

(532621, 8)
(9288, 8)


In [8]:
orders_df.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNo,StockCode
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,536365,85123A
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,71053
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,536365,84406B
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029G
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029E


In [9]:
#Descriptive statistics
orders_df.describe(include='all')


,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNo,StockCode
count,531167,532621.000000,532621,532621.000000,397924.000000,532621,532621,532621
unique,4207,NaN,19990,NaN,NaN,38,22064,4059
top,WHITE HANGING HEART T-LIGHT HOLDER,NaN,10/31/2011 14:41,NaN,NaN,United Kingdom,573585,85123A
freq,2327,NaN,1114,NaN,NaN,487622,1114,2271
mean,NaN,10.239972,NaN,3.847621,15294.315171,NaN,NaN,NaN
std,NaN,159.593551,NaN,41.758023,1713.169877,NaN,NaN,NaN
min,NaN,-9600.000000,NaN,-11062.060000,12346.000000,NaN,NaN,NaN
25%,NaN,1.000000,NaN,1.250000,13969.000000,NaN,NaN,NaN
50%,NaN,3.000000,NaN,2.080000,15159.000000,NaN,NaN,NaN
75%,NaN,10.000000,NaN,4.130000,16795.000000,NaN,NaN,NaN


# Assessing Null Values / Errors

In [15]:
#From the descriptive stats, some of the quantity and unit price amounts are less than 0
#Remove rows with that have quantity and unit price that is 0 or less
filtered_df = orders_df[(orders_df['Quantity'] >= 0) | (orders_df['UnitPrice'] >= 0)]

#filtered_df now contains only rows where both Quantity and UnitPrice are 0 or more

In [13]:
#Find null values
orders_df.isnull().sum()

Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     134697
Country             0
InvoiceNo           0
StockCode           0
dtype: int64

In [19]:
print(orders_df.shape)
print(filtered_df.shape)

(532621, 8)
(531283, 8)


In [20]:
#After some analysis, noticed that some of the StockIds have upper and lowercase values. Counting the unique stockIDs before fixing the issue
unique_stock_ids_count = orders_df['StockCode'].nunique()
print(f'Number of unique Stock IDs: {unique_stock_ids_count}')

Number of unique Stock IDs: 4059


In [21]:
# Convert all letters in StockCode to uppercase
orders_df['StockCode'] = orders_df['StockCode'].str.upper()


In [37]:
#Unique StockIDs now
count_ids2 = orders_df['StockCode'].nunique()
print(f'Number of unique Stock IDs: {count_ids2}')
print(orders_df.shape)

Number of unique Stock IDs: 3947
(532621, 8)


In [23]:
#From the descriptive stats, there are rows where quantity and unit are 0 or negative. Remove these
filtered_df = orders_df[~((orders_df['Quantity'] < 0) | (orders_df['UnitPrice'] < 0))]



In [36]:
#Unique StockIDs and Shape
count_ids3 = filtered_df['StockCode'].nunique()
print(f'Number of unique Stock IDs: {count_ids3}')
print(filtered_df.shape)

Number of unique Stock IDs: 3829
(531283, 8)


In [26]:
#Find null values
filtered_df.isnull().sum()

Description       592
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133359
Country             0
InvoiceNo           0
StockCode           0
dtype: int64

In [63]:
#Filter where CustomerId is Null
cleaned_df = filtered_df.dropna(subset=['CustomerID']).copy()

In [64]:
cleaned_df.shape

(397924, 8)

In [65]:
#See if there are any nulls left
cleaned_df.isnull().sum()

Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
InvoiceNo      0
StockCode      0
dtype: int64

In [74]:
# Reset to the pandas default setting (usually 60 rows)
pd.options.display.max_rows = 1000


In [66]:
#Replace typos in description column with most popular description by StockCode

# Find the most popular description for each StockCode
most_popular_descriptions = cleaned_df.groupby('StockCode')['Description'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else "No Description")

# Create a dictionary to map StockCode to the most popular description
description_mapping = most_popular_descriptions.to_dict()

# Fill in descriptions based on the most popular description for their StockCode
cleaned_df['Description'] = cleaned_df.apply(lambda x: description_mapping[x['StockCode']] if pd.isnull(x['Description']) or x['Description'] not in description_mapping.values() else x['Description'], axis=1)


In [67]:
#Validate one description per stock ID
unique_stock_ids_count = cleaned_df['StockCode'].nunique()
print(f'Number of unique Stock IDs: {unique_stock_ids_count}')

unique_descriptions_count = cleaned_df['Description'].nunique()
print(f'Number of unique descriptions: {unique_descriptions_count}')

Number of unique Stock IDs: 3665
Number of unique descriptions: 3647


In [68]:
# Group by Description and aggregate StockCodes into lists
grouped = cleaned_df.groupby('Description')['StockCode'].unique()

# Filter groups where the list of StockCodes has more than one element
duplicates = grouped[grouped.apply(len) > 1]

# duplicates now contains the descriptions along with the StockCodes that share those descriptions


In [69]:
#Item descriptions with multiple item numbers. This is okay.
duplicates

Description
COLOURING PENCILS BROWN TUBE                 [10133, 10135]
COLUMBIAN CANDLE RECTANGLE                   [72133, 72131]
COLUMBIAN CANDLE ROUND                       [72130, 72128]
EAU DE NILE JEWELLED PHOTOFRAME            [85023B, 85024B]
FRENCH FLORAL CUSHION COVER                  [20837, 20840]
FRENCH LATTICE CUSHION COVER                 [20835, 20838]
FROSTED WHITE BASE                           [79403, 79406]
METAL SIGN,CUPCAKE SINGLE HOOK     [82613B, 82613C, 82613A]
PAPER LANTERN 9 POINT SNOW STAR              [23041, 23040]
PINK FAIRY CAKE CUSHION COVER              [47586A, 47585A]
PINK FLOCK GLASS CANDLEHOLDER                [79329, 79331]
PINK FLOWERS RABBIT EASTER                 [35909B, 35910B]
RETRO PLASTIC 70'S TRAY                    [79190A, 79192A]
RETRO PLASTIC DAISY TRAY                   [79190D, 79191D]
RETRO PLASTIC POLKA TRAY                   [79191B, 79190B]
ROUND BLUE CLOCK WITH SUCKER               [81952B, 81953B]
SQUARE CHERRY BLOSSOM CABINE

In [70]:
#Reset index
cleaned_df=cleaned_df.reset_index(drop=True)

In [75]:
cleaned_df

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceNo,StockCode
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,536365,85123A
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,71053
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,536365,84406B
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029G
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,536365,84029E
...,...,...,...,...,...,...,...,...
397919,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,581587,22613
397920,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,581587,22899
397921,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,581587,23254
397922,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,581587,23255


## Adding in Date and Total Cost Column

In [ ]:
# Ensure 'InvoiceDate' is in datetime format
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

In [ ]:
print(df_cleaned['InvoiceDate'].dtype)

In [ ]:
# Create separate date and time columns
df_cleaned['InvoiceDate_only'] = df_cleaned['InvoiceDate'].dt.date
df_cleaned['InvoiceTime_only'] = df_cleaned['InvoiceDate'].dt.time

In [ ]:
#Change invoicedate_only to datetime
df_cleaned['InvoiceDate_only'] = pd.to_datetime(df_cleaned['InvoiceDate_only'])

# Create 'Year' and 'Month' columns 
df_cleaned['Year'] = df_cleaned['InvoiceDate_only'].dt.year
df_cleaned['Month'] = df_cleaned['InvoiceDate_only'].dt.month


In [ ]:
# Adding in TotalPrice Column using .loc for explicit indexing
df_cleaned['TotalPrice'] = df_cleaned['UnitPrice'] * df_cleaned['Quantity']


In [ ]:
df_cleaned.shape


In [ ]:
df_cleaned.head()

In [ ]:
# Identify categorical columns
categorical_columns = df_cleaned.select_dtypes(include=['object', 'category']).columns

# Get frequency counts for each categorical column
frequencies = {}
for column in categorical_columns:
    frequencies[column] = df_cleaned[column].value_counts()

# Display the frequency counts
for column, freq in frequencies.items():
    print(f"Frequency for {column}:\n{freq}\n")

# Show Sales by Top 5 Customers

In [ ]:
# Sum sales by customer
sales_by_customer = df_cleaned.groupby('CustomerID')['TotalPrice'].sum().sort_values(ascending=False)


In [ ]:
# Get the top 5 customers
top_5_customers = sales_by_customer.head(5).index.tolist()


In [ ]:
# Filter the dataset for top 5 customers
top_5_customers_data = df_cleaned[df_cleaned['CustomerID'].isin(top_5_customers)]


In [ ]:
#Convert InvoiceDate to datetime
top_5_customers_data.loc[:, 'InvoiceDate_only'] = pd.to_datetime(top_5_customers_data['InvoiceDate_only'])
#Grouping and summation
monthly_sales_top_5 = top_5_customers_data.groupby(['CustomerID', pd.Grouper(key='InvoiceDate_only', freq='ME')])['TotalPrice'].sum().reset_index()



In [ ]:
#Plot Trend lines
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# Plotting sales trends for the top 5 customers
for customer in top_5_customers:
    customer_data = monthly_sales_top_5[monthly_sales_top_5['CustomerID'] == customer]
    plt.plot(customer_data['InvoiceDate_only'], customer_data['TotalPrice'], label=customer)

plt.title('Sales Trends for Top 5 Customers')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend(title='CustomerID')
plt.xticks(rotation=45)  # Rotate date labels for better readability
plt.show()


# Clean the Description Field

In [ ]:
unique_descriptions_count = df_cleaned['Description'].nunique()
print(f'Number of unique descriptions: {unique_descriptions_count}')


In [ ]:
unique_stock_ids_count = df_cleaned['StockCode'].nunique()
print(f'Number of unique Stock IDs: {unique_stock_ids_count}')


In [ ]:
#Number of unique descriptions and unique stock ID's do not match!!!



In [ ]:
# Step 1 & 2: Group by 'StockCode' and filter
# This creates a DataFrame of groups with more than one unique 'Description'
df_filtered = df_cleaned.groupby('StockCode').filter(lambda x: x['Description'].nunique() > 1)

# Step 3: Sort the resulting DataFrame by 'StockCode' (and 'Description' if you want)
df_sorted = df_filtered.sort_values(by=['StockCode', 'Description'])

print(df_sorted[['StockCode', 'Description']])
